@YoussefM

In [13]:
import requests
from bs4 import BeautifulSoup
import lxml
import re

In [14]:
import pandas as pd

In [51]:
def poets_per_page(i: int):
    """
    takes a page number, an integer between 1 and 188, and returns the list 
    of all poets in the given page 
    """
    poets = []
    address = 'https://diwandb.com/poets?page='+str(i)
    url = requests.get(address)
    page = url.content
    soup = BeautifulSoup(page, "lxml")
    data = soup.find_all(href=re.compile("/poet/"))
    for poet in data:
        poet_url = 'https://diwandb.com'+poet.get('href')
        
        poet_page = requests.get(poet_url).content
        soup_poet = BeautifulSoup(poet_page, "lxml")
        soup_poet = soup_poet.find("div", {'class': 'col-span-1 order-last lg:order-first'})
        poet_card = soup_poet.find("div", {'class': 'overflow-y-scroll px-4 py-6 max-h-[70vh] text-sm bg-white dark:bg-gray-800 rounded shadow-lg text-base'})
        poet_name = soup_poet.find("h2", {"class": "my-4"}).text
        poet_period = soup_poet.find(href=re.compile("period"))
        if poet_period != None:
            poet_period = poet_period.text.strip("#")
        poets += [[poet_name, poet_card.text.strip("\n"), poet_period, poet_url]]
        
        
    return poets

In [52]:
# get the list of all poets in the website, that's 4,512 poet. 
from tqdm.notebook import tqdm
all_poets = []
for i in tqdm(range(1,189)):
    all_poets += poets_per_page(i)

  0%|          | 0/188 [00:00<?, ?it/s]

We'll create a csv file to contain the poets with their biographie.

In [ ]:
###pd.DataFrame(all_poets).to_csv('all_poets')

In [139]:
df = pd.read_csv("all_poets")

In [140]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [141]:
df.rename(columns={'0':'poet', '1':'poet_bio', '2':'period', '3':'poet_url'}, inplace=True)

In [142]:
# the biography of Ibn Rumi for example (https://en.wikipedia.org/wiki/Ibn_al-Rumi)
df['poet_bio'][0]

'ابن الرومي221 - 283 هـ / 836 - 896 معلي بن العباس بن جريج أو جورجيس، الرومي.شاعر كبير، من طبقة بشار والمتنبي، رومي الأصل، كان جده من موالي بني العباس.ولد ونشأ ببغداد، ومات فيها مسموماً قيل: دس له السمَّ القاسم بن عبيد الله -وزير المعتضد- وكان ابن الرومي قد هجاه.قال المرزباني: لا أعلم أنه مدح أحداً من رئيس أو مرؤوس إلا وعاد إليه فهجاه، ولذلك قلّت فائدته من قول الشعر وتحاماه الرؤساء وكان سبباً لوفاته.وقال أيضاً: وأخطأ محمد بن داود فيما رواه لمثقال (الوسطي) من أشعار ابن الرومي التي ليس في طاقة مثقال ولا أحد من شعراء زمانه أن يقول مثلها إلا ابن الرومي.'

The next two functions will be used to define `poems_per_poet(poet_url)` below

In [68]:
def get_titles_per_page(address):
    titles_per_page = []
    url = requests.get(address)
    page = url.content
    soup = BeautifulSoup(page, "lxml")
    # list of all the titles in the page (address)
    frames = soup.find_all("div", {'class': 'slice'})
    for i in range(len(frames)):
        titles_per_page += ['https://diwandb.com'+frames[i].find(href=re.compile("/poem/")).get('href')]
    return titles_per_page

# the output of this function will be fed to get_poem_info()  

In [118]:
get_titles_per_page('https://diwandb.com/poet/%D8%B9%D9%84%D9%8A-%D8%A8%D8%A7.html?page=2')[0:5]

['https://diwandb.com/poem/أحد-عشر-كوكبا.html',
 'https://diwandb.com/poem/ليلة-الغرباء.html',
 'https://diwandb.com/poem/فبدأت-حيث-توقفوا.html',
 'https://diwandb.com/poem/أسير-هموم.html',
 'https://diwandb.com/poem/فقد-الأحبة-غربة.html']

In [69]:
def pages_per_poet(poet_url: str)->list:
    """
    Given a poet, this function will return all the pages of his/her poems
    """
    all_pages = []
    url = requests.get(poet_url.split('?')[0])
    page = url.content
    soup = BeautifulSoup(page, "lxml")
    pages = soup.find("nav", {'class': 'pagination'})
    if pages==None:
        return poet_url
    i = 0
    while pages.find('button', {'wire:click': "nextPage"}):
        poet_url = poet_url.split('?')[0] + f"?page={i+1}"
        all_pages += [poet_url]
        url = requests.get(poet_url)
        page = url.content
        soup = BeautifulSoup(page, "lxml")
        pages = soup.find("nav", {'class': 'pagination'})
        i +=1
    return all_pages
    
# the output of this function will be fed to get_titles_per_page()

This is an important function. Given the url of a poet, it returns the list of his/her poems

In [70]:
def poems_per_poet(poet_url):
    """
    returns the poems of a given author
    """
    poems = []
    for address in pages_per_poet(poet_url):
        poems += get_titles_per_page(address)
    return poems

As an example we will retrieve the titles of the poems of `Abou Alaa al-Maarri (December 973 – May 1057)` (https://en.wikipedia.org/wiki/Al-Ma%27arri). There are `1869` of his poems in the database of Diwandb.

In [82]:
poet_url = 'https://diwandb.com/poet/أبو-العلاء-المعري.html'
poems = poems_per_poet(poet_url)

In [86]:
len(poems)

'https://diwandb.com/poem/قد-كنت-أحيانا-شديد-المعتمد.html'

Feeding all_poets to the function `poems_per_poet(poet_url)` above, we get all the (urls of) poems published in the website. Then to each of these, we apply the next function to get the  poem itself, its `period` and `meter`. 

In [87]:
def get_poem_info(poem_url: str)->list:
    """
    the function gets the url address and returns the info of the poem
    """
    url = requests.get(poem_url)
    page = url.content
    soup = BeautifulSoup(page, "lxml")
    # the body of the poem
    poem = soup.find("div", {'class': 'mx-auto w-full text-xl sm:text-3xl md:max-w-lg twice leading-12'})
    # the title
    title = soup.find("div", {'class': 'mb-6'}).find("h1").text
    # next we extract the name of the poet, the meter and the historical period
    data = soup.find("div", {'class': 'flex flex-wrap justify-between items-center my-4 leading-10'})
    if poem == None:
        poem = soup.find("div", {'class': 'clear-both w-full text-xl sm:text-3xl text-center leading-12'})
   
    poem = poem.contents[1:-1]
    #poet = data.find(href=re.compile("poet")).text.strip('\n')
    meter = data.find(href=re.compile("meter"))
    if meter:
        meter = meter.text.strip('#')
    # uncomment period if you want to include it in your function
    #period = data.find(href=re.compile("period")).text.strip('#')
    return title, poem, meter

Now we download the `1869` poem of `Abou Alaa al Maarri`

In [88]:
abou_alaa = []
for poem in tqdm(poems):
    abou_alaa += [get_poem_info(poem)]

  0%|          | 0/1869 [00:00<?, ?it/s]

Let's save the poems locally into a pandas dataframe

In [156]:
###pd.DataFrame(abou_alaa).to_csv('abou_alaa')

We load the poems for exploration

In [180]:
abou_alaa = pd.read_csv('abou_alaa')

In [181]:
abou_alaa.drop('Unnamed: 0', axis=1, inplace=True)

In [182]:
abou_alaa.rename(columns={'0':'title','1':'poem','2':'meter'}, inplace=True)

In [201]:
abou_alaa

,title,poem,meter
0,قصيدة توَهّمتُ خَيراً في الزّمانِ وأهلِهِ،,[<p>تَوَهَّمتُ خَيراً في الزَمانِ وَأَهلِهِ</p...,بحر الطويل
1,قصيدة لعلّ أُناساً، في المحاريبِ، خَوّفوا,[<p>لَعَلَّ أُناساً في المَحاريبِ خَوَّفوا</p>...,بحر الطويل
2,قصيدة تحمّلْ عن أبيكَ الثّقلَ، يوماً،,"[<p>تَحَمَّل عَن أَبيكَ الثِقلَ يَوماً</p>, <p...",بحر الوافر
3,قصيدة أُولو الفضلِ، في أوطانهم، غرباءُ،,"[<p>أُولو الفضلِ، في أوطانهم، غرباءُ،</p>, <p>...",NaN
4,قصيدة غدَوتَ مريضَ العقلِ والدّينِ فالقَني,[<p>غَدَوتَ مَريضَ العَقلِ وَالدينِ فَاِلقَني<...,بحر الطويل
...,...,...,...
1864,قصيدة تغدو على الأرض في حالات ساكنها,"[<p>تَغدو عَلى الأَرضِ في حالاتِ ساكِنِها</p>,...",بحر البسيط
1865,قصيدة ألم يَبْلُغْكَ فَتْكي بالمَواضي,"[<p>ألم يَبْلُغْكَ فَتْكي بالمَواضي</p>, <p>وس...",NaN
1866,قصيدة أيَدْفَعُ مُعْجِزاتِ الرُّسْل قوْمٌ,"[<p>أيَدْفَعُ مُعْجِزاتِ الرُّسْل قوْمٌ</p>, <...",NaN
1867,قصيدة ألحُسْنُ يَعْلَمُ أنّ مَنْ وارَيْتَهُ,"[<p>ألحُسْنُ يَعْلَمُ أنّ مَنْ وارَيْتَهُ</p>,...",NaN


Let's for example check the second poem in the list 

In [200]:
abou_alaa['poem'][1].split(',')

['[<p>لَعَلَّ أُناساً في المَحاريبِ خَوَّفوا</p>',
 ' <p>بِآيٍ كَناسٍ في المَشارِبِ أَطرَبوا</p>',
 ' <p>إِذا رامَ كَيداً بِالصَلاةِ مُقيمُها</p>',
 ' <p>فَتارِكُها عَمداً إِلى اللَهِ أَقرَبُ</p>',
 ' <p>فَلا يُمسِ فَخّاراً مِنَ الفَخرِ عائِدٌ</p>',
 ' <p>إِلى عُنصُرِ الفَخّارِ لِلنَفعِ يُضرَبُ</p>',
 ' <p>لَعَلَّ إِناءً مِنهُ يُصنَعُ مَرَّةً</p>',
 ' <p>فَيَأكُلُ فيهِ مَن أَرادَ وَيَشرَبُ</p>',
 ' <p>وَيُحمَلُ مِن أَرضٍ لِأُخرى وَما دَرى</p>',
 ' <p>فَواهاً لَهُ بَعدَ البِلى يَتَغَرَّبُ</p>]']